In [1]:
from config import cfg, get_data_dir, get_output_dir


In [2]:
from easydict import EasyDict as edict
from edgeConstruction import compressed_data
import matplotlib.pyplot as plt
import data_params as dp
import make_data
import pretraining
import extract_feature
import copyGraph
import DCC

/Users/jasper/opt/miniconda3/envs/mmc/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
import os
import numpy as np
import scipy.io as sio

In [4]:
from custom_data import DCCPT_data

# load data & make graph

## remove labels if necessary

In [5]:
datadir = get_data_dir("test")
outputdir = get_output_dir("test")

In [65]:
traindata = sio.loadmat(os.path.join(datadir, "traindata.mat"))

In [68]:
testdata = sio.loadmat(os.path.join(datadir, "testdata.mat"))

In [67]:
traindata['X'].shape

(60000, 784)

In [69]:
testdata['X'].shape

(10000, 784)

In [72]:
mnist_data = np.vstack([traindata['X'], testdata['X']])

In [73]:
mnist_data.shape

(70000, 784)

In [74]:
datadir

'/Users/jasper/dev/DCC/data/test'

In [75]:
sio.savemat(os.path.join(datadir, "mnistdata.mat"), {'X': mnist_data})

## load data

In [6]:
mnistdata = sio.loadmat(os.path.join(datadir, "mnistdata.mat"))

In [7]:
X = mnistdata['X']

In [8]:
X.shape

(70000, 784)

## make graph

In [9]:
from edgeConstruction import mkNN

In [11]:
dict_params_make_graph = {
    'k': 50,
    'preprocess': None,
    'algo': 'mknn',
    'measure': 'cosine',
    'isPCA': None,
}


In [14]:
n_samples = 10000

In [15]:
X_graph = mkNN(X[:n_samples], k=dict_params_make_graph['k'], measure=dict_params_make_graph['measure'])

In [17]:
X_graph.shape

(133001, 3)

In [18]:
X_graph

array([[1.00000000e+00, 2.10000000e+01, 1.00000000e+00],
       [3.00000000e+00, 2.30000000e+01, 1.02345058e+00],
       [7.00000000e+00, 2.70000000e+01, 1.00000000e+00],
       ...,
       [9.22400000e+03, 9.99900000e+03, 1.00000000e+00],
       [9.84300000e+03, 9.99900000e+03, 1.00000000e+00],
       [9.96900000e+03, 9.99900000e+03, 1.00000000e+00]])

In [20]:
sio.savemat(os.path.join(datadir, "pretrained.mat"), mdict={'X': X[:n_samples], 'w': X_graph[:, :2]})


# pretraining SDAE